In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Embedding, LeakyReLU, \
                        Merge, concatenate, Input, BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.preprocessing import text, sequence
from sklearn.utils import shuffle
import keras
import gensim 

Using TensorFlow backend.


In [2]:
trainvar = pd.read_csv('training_variants')
traintex = pd.read_csv('training_text', sep='\|\|', header=None, names=['ID','Text'], skiprows=[0])
train = pd.merge(trainvar, traintex, on='ID')

trainvar = pd.read_csv('test_variants')
traintex = pd.read_csv('test_text', sep='\|\|', header=None, names=['ID','Text'], skiprows=[0])
test = pd.merge(trainvar, traintex, on='ID')

/home/arun/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/arun/Programs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
train.Text = train.Gene +' '+ train.Variation +' '+ train.Text
test.Text = test.Gene + test.Variation + test.Text
train.Text = train.Text.str.lower()
test.Text = test.Text.str.lower()
for words in stopwords.words('english'):
    test.Text = test.Text.str.replace(' '+words+' ', '')
    train.Text = train.Text.str.replace(' '+words+' ', '')

In [5]:
traintext = test.Text.tolist()+train.Text.tolist()
sentences = []
for t in traintext:
    #for s in sent_tokenize(t):
    sentences.append(text.text_to_word_sequence(t))

In [6]:
maxlength=10000
vec_size=50
vectorizer = gensim.models.Word2Vec(sentences, size=vec_size, sg=1)

In [8]:
y = keras.utils.to_categorical(train['Class']-1, num_classes=9)
tok = text.Tokenizer()
tok.fit_on_texts(train['Text'])
X = tok.texts_to_sequences(train['Text'])
X = sequence.pad_sequences(X ,maxlen=maxlength)

In [9]:
weights = np.zeros(shape=(len(tok.word_index)+1,vec_size))
for k, v in tok.word_index.items():
    if k in vectorizer.wv.vocab:
        weights[v] = vectorizer.wv[k]      
weights = weights/np.amax(abs(weights))

In [10]:
# model = Sequential()
# model.add(Embedding(max(tok.word_index.values())+1, 15, input_length=maxlength))
# model.add(Conv1D(128,10,activation='relu'))
# model.add(MaxPooling1D())
# model.add(Dropout(0.2))
# model.add(Conv1D(128,10,activation='relu'))
# model.add(MaxPooling1D())
# model.add(Flatten())
# model.add(Dense(128,activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(units=9,activation='softmax'))


input1 = Input(shape=(maxlength,))
x = Embedding(max(tok.word_index.values())+1, vec_size, input_length=maxlength, weights=[weights])(input1)
x = Conv1D(128,10, activation='relu')(x)
#x = LeakyReLU(0.1)(x)
x = BatchNormalization()(x)
x = MaxPooling1D()(x)
x = Dropout(0.2)(x)
x = Conv1D(128,10, activation='relu')(x)
x = BatchNormalization()(x)
#x = LeakyReLU(0.1)(x)
x = MaxPooling1D()(x)
x = Flatten()(x)
x = Dense(128)(x)
x = LeakyReLU(0.1)(x)
x = BatchNormalization()(x)
output = Dense(units=9, activation='softmax')(x)
model = Model(inputs=input1, outputs=output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 5000, 50)          58864600  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4991, 128)         64128     
_________________________________________________________________
batch_normalization_1 (Batch (None, 4991, 128)         512       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2495, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2495, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2486, 128)         163968    
__________

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=3, verbose=1, validation_split=.05)

Train on 3154 samples, validate on 167 samples
Epoch 1/3
3154/3154 [==============================] - 296s - loss: 0.7557 - acc: 0.7698 - val_loss: 2.9888 - val_acc: 0.1796
Epoch 2/3
3154/3154 [==============================] - 296s - loss: 0.5201 - acc: 0.8272 - val_loss: 2.7476 - val_acc: 0.1916
Epoch 3/3
3154/3154 [==============================] - 298s - loss: 0.4251 - acc: 0.8386 - val_loss: 2.5201 - val_acc: 0.1796


In [12]:
model.fit(X, y, batch_size=128, epochs=1, verbose=1)

Epoch 1/1
3321/3321 [==============================] - 299s - loss: 0.5344 - acc: 0.8193    


In [13]:
model.save('wordvec.h5')

In [14]:
x = tok.texts_to_sequences(test['Text'])

In [15]:
classes = model.predict(sequence.pad_sequences(x,maxlen=maxlength), batch_size=32, verbose=1)

5668/5668 [==============================] - 193s   


In [16]:
cols=['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7', 'class8', 'class9']
dfsub = pd.DataFrame(classes, columns=cols)
dfsub.insert(0,'ID',range(len(dfsub.class1)))
dfsub.head(15)

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,0,7.643345e-11,0.006044,1.044639e-06,0.001856,0.000319,0.000004,0.991706,0.000001,0.000069
1,1,7.862521e-10,0.007780,2.613663e-06,0.008557,0.000815,0.000013,0.982644,0.000004,0.000184
2,2,4.481734e-11,0.004419,8.306049e-07,0.002551,0.000338,0.000003,0.992622,0.000001,0.000066
3,3,4.760287e-11,0.004211,8.806072e-07,0.001826,0.000274,0.000003,0.993619,0.000001,0.000066
4,4,1.154017e-10,0.005105,1.252988e-06,0.004873,0.000491,0.000004,0.989428,0.000002,0.000096
5,5,7.590027e-10,0.008389,2.254199e-06,0.009819,0.000836,0.000014,0.980769,0.000004,0.000167
6,6,3.427970e-10,0.007531,1.724295e-06,0.005370,0.000581,0.000008,0.986385,0.000003,0.000120
7,7,1.556621e-10,0.006554,1.391429e-06,0.004764,0.000501,0.000005,0.988077,0.000002,0.000096
8,8,4.765874e-10,0.007517,2.027721e-06,0.005835,0.000581,0.000010,0.985914,0.000003,0.000138
9,9,3.491654e-11,0.005038,7.190160e-07,0.001779,0.000300,0.000002,0.992822,0.000001,0.000057


In [17]:
dfsub.to_csv('wordvec.csv',index=False)